In [3]:
!pip install -q lens-metric

In [4]:
import pandas as pd
import json
import torch


In [5]:
from lens import download_model, LENS, LENS_SALSA
lens_salsa_path = download_model("davidheineman/lens-salsa") # see https://huggingface.co/davidheineman/lens-salsa
lens_salsa = LENS_SALSA(lens_salsa_path)

complex_text = [
    "They are culturally akin to the coastal peoples of Papua New Guinea."
]
simple_text = [
    "They are culturally similar to the people of Papua New Guinea."
]

scores, word_level_scores = lens_salsa.score(complex_text, simple_text, batch_size=8, devices=[0])
print(scores) # [72.40909337997437]

# LENS-SALSA also returns an error-identification tagging, recover_output() will return the tagged output
tagged_output = lens_salsa.recover_output(word_level_scores, threshold=0.5)
print(tagged_output)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning_fabric/utilities/cloud_io.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_locati

[72.40909337997437]
[{'src_tagged': 'they are culturally akin to the coastal peoples of papua new guinea.', 'mt_tagged': 'they are culturally similar to the people of papua new guinea.'}]


In [6]:
complex_filepath = '/content/LLaMa 3.1 70B instruction (2nd run)'
simple_filepath = '/content/test.json'

In [7]:
import json
with open(complex_filepath,'r', encoding='utf-8') as f:
  complex = json.load(f)
with open(simple_filepath,'r', encoding='utf-8') as s:
  simple = json.load(s)

In [8]:
complex_text = []
simple_text = []
for q_key, q_data in complex.items():
  for abstract_key, abstract_data in q_data.get("abstracts", {}).items():
    sentences = abstract_data.get("sentences", [])
    complex_text += sentences

for q_key, q_data in simple.items():
  for abstract_key, abstract_data in q_data.get("abstracts", {}).items():
    sentences = abstract_data.get("sentences", [])
    simple_text += sentences


In [9]:
assert len(complex_text) == len(simple_text)
print(len(complex_text))
# print(complex_text[:3])
# print(simple_text[:3])
for a,b in zip(complex_text,simple_text):
  assert len(a) != 0 and len(b) != 0
  if len(a)<=3:
    print(a)
  if len(b)<=3:
    print(b)

4060


In [10]:
def longest_five_string_lengths(str_list):
    # 计算每个字符串的单词数量
    word_counts = [(s, len(s.split())) for s in str_list]

    # 按单词数量降序排列，并取前五个
    longest_five = sorted(word_counts, key=lambda x: x[1], reverse=True)[:5]

    # 返回前五个字符串的单词数量
    return [count for _, count in longest_five]

In [11]:
longest_5_comp = longest_five_string_lengths(complex_text)
longest_5_simp = longest_five_string_lengths(simple_text)
print(longest_5_comp)
print(longest_5_simp)

[246, 242, 218, 209, 199]
[111, 94, 88, 87, 87]


In [12]:
for a in complex_text:
  if len(a.split()) >100:
    a = a[:100]

In [13]:
from lens import download_model, LENS, LENS_SALSA
lens_salsa_path = download_model("davidheineman/lens-salsa") # see https://huggingface.co/davidheineman/lens-salsa
lens_salsa = LENS_SALSA(lens_salsa_path)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning_fabric/utilities/cloud_io.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_locati

In [14]:
import torch
from lens import download_model, LENS, LENS_SALSA

# ... (your existing code) ...

# Function to truncate or pad sequences to a fixed length
def process_sequences(sequences, max_length):
    processed_sequences = []
    for seq in sequences:
        if len(seq) > max_length:
            # Truncate if sequence is too long
            processed_sequences.append(seq[:max_length])
        else:
            # Pad if sequence is too short
            processed_sequences.append(seq)  # Pad with empty strings
    return processed_sequences

# Define maximum sequence length
max_length = 512  # Or any suitable value based on your model and data

# Process complex_text and simple_text
complex_text = process_sequences(complex_text, max_length)
simple_text = process_sequences(simple_text, max_length)

# Now call the lens_salsa.score function with the processed sequences
scores, word_level_scores = lens_salsa.score(complex_text, simple_text, batch_size=8, devices=[0])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 508/508 [02:22<00:00,  3.56it/s]


In [15]:
# scores, word_level_scores = lens_salsa.score(complex_text, simple_text, batch_size=8, devices=[0])
print(scores)

[68.88090968132019, 64.48931097984314, 70.99950909614563, 60.964787006378174, 32.92659521102905, 39.71706032752991, 30.44680655002594, 45.793819427490234, 43.60455572605133, 65.22794365882874, 50.9704053401947, 57.65597224235535, 63.62724304199219, 45.42309641838074, 63.33879232406616, 76.48494243621826, 52.93872356414795, 67.18837022781372, 66.04161262512207, 62.56789565086365, 70.17924189567566, 65.09935855865479, 68.37354302406311, 60.524338483810425, 72.33685255050659, 75.74440240859985, 55.520325899124146, 69.53964233398438, 61.706072092056274, 72.22475409507751, 71.67079448699951, 17.579789459705353, 49.706822633743286, 70.73473930358887, 75.7891833782196, 63.497769832611084, 71.37758135795593, 65.31375646591187, 46.10448777675629, 80.49217462539673, 58.568257093429565, 64.58566188812256, 45.43800354003906, 30.492910742759705, 62.842488288879395, 68.31097602844238, 69.22249794006348, 67.08220839500427, 62.782448530197144, 72.97266721725464, 68.20897459983826, 57.82323479652405, 3

In [17]:
# prompt: create a dataframe with column [complex_text, simple_text, scores]

import pandas as pd

# Assuming complex_text, simple_text, and scores are already defined from your previous code
# Example data (replace with your actual data)
data = {'complex_text': complex_text, 'simple_text': simple_text, 'scores': scores}

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df['scores'].mean()

61.78977134743054

In [16]:
import numpy as np
import scipy.stats as stats

mean = np.mean(scores)
std_dev = np.std(scores,ddof=1)
n = len(scores)
t_score = stats.t.ppf(0.975, df=n-1)  # 0.975 for 95% CI (two-tailed)
standard_error = std_dev / np.sqrt(n)
margin_of_error = t_score * standard_error
print(f"The 95% confidence interval is: ({mean}+-{margin_of_error})")


The 95% confidence interval is: (61.78977134743054+-0.30341066046188475)
